In [ ]:
require 'image';

In [ ]:
left = image.load('data/kitti_stereo/training/image_0/000127_10.png');
right = image.load('data/kitti_stereo/training/image_1/000127_10.png');
disp = image.load('data/kitti_stereo/training/disp_noc/000127_10.png', 1, 'byte');
itorch.image(left);
itorch.image(right);
itorch.image(disp);

apxRight = torch.Tensor(left:size())
apxRight:fill(0)
for y = 1, left:size(2) do
    for x = 1, left:size(3) do
        local idx = math.min(x+disp[1][y][x], left:size(3))
        if idx ~= x then
          apxRight[1][y][idx] = left[1][y][x]
        end
    end
end

itorch.image(apxRight);

--diff = right:type('torch.FloatTensor') - apxRight:type('torch:FloatTensor')
--itorch.image(diff)

--print(left:size(), right:size(), disp:size());